# **08-01 합성곱과 풀링(Convolution and Pooling)**

- https://wikidocs.net/62306
- 합성곱 신경망(CNN, Convolutional Neural Network)은 이미지 처리에 탁월한 성능을 보이는 신경망.
- CNN은 크게 **합성곱층(Convolution layer)** 과 **풀링층(Pooling layer)** 으로 구성된다.  
    - **CONV**: 합성곱 연산 (커널을 이용해 특징 추출) - 합성곱층
    - **ReLU**: 비선형성 추가 (활성화 함수) - 합성곱층
    - **POOL**: 풀링 연산 (특징맵 크기 축소, 다운샘플링) - 풀링층

## 1. 합성곱 신경망의 대두

### (1) MLP의 한계
- 이미지를 **다층 퍼셉트론(MLP)** 로 처리하려면 1차원 벡터로 변환해야 함.  
- 이 과정에서 **공간적 구조(spatial structure)** 정보가 손실됨.  
- 손글씨 ‘Y’를 예로 들면, 같은 글자라도 **휘어짐/이동/회전** 등으로 픽셀 값이 달라지면 MLP는 잘 인식하지 못함.

### (2) 예시
- 알파벳 `Y` 손글씨 (정자로 쓴 것 vs 휘갈겨 쓴 것):  
  - **사람**: 쉽게 둘 다 `Y`라고 인식 가능.  
  - **기계(MLP)**: 픽셀 값 차이가 크기 때문에 전혀 다른 입력으로 인식.  

### (3) 문제점
- 2차원 이미지를 1차원으로 변환하면 원래의 공간적 구조가 사라짐.  
  - 예: 가까운 픽셀 간 연관성, 비슷한 값 간의 관계 등이 유실됨.  
- 결국 **이미지의 공간적 구조를 보존**하면서 학습할 수 있는 새로운 구조가 필요했음.

### (4) 해결책
- CNN은 **합성곱 연산**을 통해 **지역적 특징(패턴)** 을 추출하고,  
- **공간적 구조 정보를 유지**하면서 학습 가능.  
- 따라서 이미지/비전 분야에서 탁월한 성능을 보임.

### (5) 요약
- MLP는 이미지의 **위치/형태 변화**에 약하고 공간 구조를 잃는다.  
- CNN은 **합성곱 연산**으로 공간 구조를 보존하면서 특징을 뽑아내기 때문에 이미지 인식에 강하다.  

## 2. 채널 (Channel)

이미지는 **(높이, 너비, 채널)** 형태의 **3차원 텐서**로 표현된다.  
- **높이(Height)**: 세로 방향 픽셀 수  
- **너비(Width)**: 가로 방향 픽셀 수  
- **채널(Channel)**: 색 성분(컬러 정보)

### (1) 흑백 이미지
- 채널 수 = 1  
- 각 픽셀 값은 **0 ~ 255** 범위의 명암 값  
- 예: `28 × 28` 픽셀 손글씨 이미지 → `(28 × 28 × 1)` 크기의 텐서

### (2) 컬러 이미지
- 채널 수 = 3 (**RGB: Red, Green, Blue**)  
- 하나의 픽셀 = 세 가지 색(삼원색)의 조합  
- 예: `28 × 28` 픽셀 컬러 이미지 → `(28 × 28 × 3)` 크기의 텐서

### (3) 깊이(Depth)
- 채널(Channel)은 종종 **깊이(Depth)** 라고도 부른다.  
- 따라서 이미지를 **(높이, 너비, 깊이)** 형태의 3차원 텐서로 표현할 수 있음.
  
### (4) 요약
- 흑백 이미지는 `(H × W × 1)`  
- 컬러 이미지는 `(H × W × 3)`  
- CNN은 이 **채널(=깊이)** 정보를 이용해 이미지의 색상 및 패턴을 학습한다.

## 3. 합성곱 연산 (Convolution Operation)

합성곱층은 이미지를 입력받아 **특징을 추출하는 역할**.  
합성곱(Convolution)이란 **커널(kernel) 또는 필터(filter)** 라는 작은 행렬을 이미지 위에 겹쳐가며 훑으면서, 겹치는 부분끼리 곱한 뒤 모두 더한 값을 출력으로 내는 연산을 함.  

- 커널 크기는 보통 `3×3` 또는 `5×5`를 사용.  
- 합성곱 연산 결과를 **특성 맵(Feature Map)** 이라고 부름.  

### (1) 합성곱 연산 과정
- 커널을 입력 이미지 위에 겹쳐 이동시키며 연산을 수행  
- 한 번의 위치 이동 = **1 스텝(step)**

### (2) 스트라이드 (Stride)
- 커널이 한 번 이동할 때 건너뛰는 칸 수  
- 예: 스트라이드 = 1 → 한 칸씩 이동  
- 예: 스트라이드 = 2 → 두 칸씩 이동 → 출력 특성 맵 크기가 더 작아짐  

예시:  
- `5×5` 이미지 + `3×3` 커널  
- 스트라이드 = 2 → 최종 출력 특성 맵 크기 = `2×2`

### (4) 요약
- 합성곱 연산 = 이미지 ⊗ 커널 → **특성 맵** 생성  
- **커널 크기**: 3×3, 5×5 주로 사용  
- **스트라이드**: 커널 이동 간격 → 값이 커질수록 출력 크기 작아짐

## 4. 패딩 (Padding)

합성곱 연산을 수행하면 입력보다 작은 크기의 **특성 맵**이 생성됨.  
예: `5×5` 이미지에 `3×3` 커널, 스트라이드 = 1 -> 결과는 `3×3` 특성 맵이 됨.  
이처럼 합성곱을 여러 번 반복하면 출력 크기가 계속 줄어드는 문제가 발생함.  

이를 해결하기 위해 **패딩(Padding)** 을 사용함.  
→ 입력의 가장자리에 지정된 폭만큼 행과 열을 추가하여 테두리를 확장하는 방식임.  

### (1) 제로 패딩 (Zero Padding)
- 가장 일반적으로 사용하는 방식  
- 입력 테두리를 0으로 채움  
- 예: `5×5` 이미지에 폭 1짜리 제로 패딩 → `7×7` 이미지로 확장됨  

### (2) 커널 크기와 패딩
- 커널이 `3×3`, 스트라이드 = 1 → **패딩 1** 적용 시 출력 크기 보존 가능  
- 커널이 `5×5`, 스트라이드 = 1 → **패딩 2** 적용 시 출력 크기 보존 가능  

예:  
- `5×5` 입력 + 패딩 1 적용 → `7×7` 이미지  
- `3×3` 커널, 스트라이드 1로 합성곱 -> 결과 크기 `5×5` (입력과 동일)

### (3) 요약
- 합성곱 연산은 입력 크기를 줄이는 특징이 있음.  
- 패딩은 입력 가장자리에 여백을 추가하여 **출력 크기를 보존하는 역할**을 함.  
- 주로 **제로 패딩**을 사용함.  

## 5. 가중치와 편향 (Weights & Bias)

### (1) 합성곱 신경망의 가중치
- **MLP**
    - MLP에서 `3×3` 이미지를 처리한다고 가정하면 → 입력층 뉴런 수 = 9개  
    - 은닉층에 뉴런 4개가 있을 경우 → 총 가중치 수 = `9×4 = 36개`가 됨.  
- **CNN**
    - CNN에서 `2×2` 커널을 사용한다고 하면  
    - CNN의 가중치는 **커널 행렬의 원소 값들** 이므로 필요한 가중치는 `2×2 = 4개`.
    - CNN은 커널을 이미지 전체에 동일하게 적용하므로, **훨씬 적은 가중치로 공간적 구조 정보를 보존**할 수 있음.  
- 즉, CNN은 **적은 수의 가중치**로도 효과적인 학습을 수행함.  

### (2) 활성화 함수
- CNN에서도 합성곱 연산 후에는 **비선형성**을 추가하기 위해 활성화 함수를 통과시킴.  
- 주로 **ReLU 함수**나 그 변형을 사용함.  
- 합성곱 연산 + 활성화 함수를 거친 결과를 **합성곱 층(Convolution Layer)** 이라고 부름.

### (3) 합성곱 신경망의 편향
- CNN에서도 **편향(Bias)** 를 사용 가능.  
- 커널 적용 후 결과값에 **편향을 더함**.  
- 편향은 하나의 값만 존재하며, 해당 커널로 생성된 특성 맵 전체에 동일하게 더해짐.

### (4) 요약
- CNN의 가중치는 **커널의 원소 값**으로, MLP보다 훨씬 적음.  
- CNN은 **공간적 구조를 보존**하며 학습할 수 있음.  
- 합성곱 층은 **합성곱 연산 + 활성화 함수(ReLU 등)**로 구성됨.  
- 편향은 **하나의 값**으로, 특성 맵 전체에 더해짐.  

## 6. 특성 맵의 크기 계산 방법

- 입력의 높이와 너비, 커널의 높이와 너비, 스트라이드 값을 이용해 특성 맵 크기를 구할 수 있음.  
- 계산 과정에서 소수점이 발생하면 버림 처리함.
- 패딩을 고려할 경우 패딩 폭까지 포함해서 계산함.
- 스트라이드가 커질수록 출력 크기는 작아짐.  
- 패딩을 사용하면 입력과 출력 크기를 동일하게 유지할 수 있음.  

### 예시

- 입력: `5×5` 이미지  
- 커널: `3×3`  
- 스트라이드: 1  
- 결과: 특성 맵 크기 = `3×3`  
- 이는 총 9번의 합성곱 스텝이 수행됨을 의미함.  

### 패딩 고려

- 패딩을 적용하면 입력 테두리에 여백이 추가되므로 결과 크기가 달라짐.  
- 예: `5×5` 입력에 패딩 1을 적용 → `7×7` 입력으로 확장됨  
- 여기에 `3×3` 커널, 스트라이드 1을 적용하면 → 출력은 다시 `5×5`가 됨.  

## 7. 다수의 채널을 가질 경우의 합성곱 연산(3차원 텐서의 합성곱 연산)

- **다수의 채널을 가진 이미지**
- 입력 데이터가 여러 채널을 가진 경우, **커널 또한 동일한 채널 수**를 가져야 함.  
- 즉, 입력 채널 수 = 커널 채널 수 이어야 함.  
- 합성곱 연산은 채널별로 수행한 뒤, 결과를 모두 합산하여 최종 특성 맵을 얻음. 

### (1) 예시
- 입력: 높이 3, 너비 3, 채널 3 (예: RGB 이미지)  
- 커널: 높이 2, 너비 2, 채널 3  
- 연산 결과: 높이 2, 너비 2, 채널 1의 특성 맵이 생성됨.  
- 이때 주의할 점: 커널은 3개의 커널이 아니라, **3채널을 가진 1개의 커널**

### (2) 특성 맵의 채널 의미
- 합성곱 연산 결과로 얻는 특성 맵의 채널은 **RGB 같은 색상 의미를 갖지 않음**.  
- 대신, CNN이 학습 과정에서 자동으로 추출한 **특징 차원(feature dimension)** 을 의미함.

## 8. 3차원 텐서의 합성곱 연산

### (1) 입력과 커널
- 입력은 **높이, 너비, 채널**을 가짐.  
- 커널도 동일한 채널 수를 가져야 함.  
- 따라서 입력 채널 수 = 커널 채널 수 이어야 함.  

### (2) 연산 결과
- 입력과 커널이 합성곱 연산을 하면 → **특성 맵(출력)** 이 생성됨.  
- 이때 하나의 커널은 **출력 채널 1개**를 만들어냄.  

### (3) 다수의 커널 사용
- 합성곱 연산에 여러 개의 커널을 사용 가능함.  
- 커널 개수 = 출력 특성 맵의 채널 수가 됨.  
- 즉, 커널을 N개 사용하면 출력 특성 맵의 채널도 N개가 됨.  

### (4) 가중치 매개변수 개수
- 가중치는 커널의 원소들임.  
- 하나의 커널은 **(커널 높이 × 커널 너비 × 입력 채널 수)** 개의 가중치를 가짐.  
- 커널이 여러 개일 경우, 모든 커널의 가중치를 합산한 값이 전체 매개변수 수가 됨.  

## 9. 풀링 (Pooling)

- 합성곱 층(합성곱 연산 + 활성화 함수) 뒤에는 보통 **풀링 층**을 추가함.  
- 풀링 층은 **특성 맵을 다운샘플링하여 크기를 줄이는 역할**

### (1) 풀링의 개념
- **커널과 스트라이드** 개념을 가짐.  
- 커널이 겹치는 영역 내에서 값을 집계하여 더 작은 크기의 특성 맵을 만듦.  
- 학습해야 할 가중치가 없음.  
- 연산 후에도 **채널 수는 변하지 않음**.  

### (2) 풀링의 종류
1. **최대 풀링 (Max Pooling)**  
   - 커널이 덮는 영역 내에서 **최댓값**을 추출함.  
   - 가장 뚜렷한 특징을 보존하는 효과가 있음.  
   - 예: `2×2` 커널, 스트라이드 2 → 특성 맵 크기가 절반으로 줄어듦.  
2. **평균 풀링 (Average Pooling)**  
   - 커널이 덮는 영역 내에서 **평균값**을 추출함.  
   - 특징을 부드럽게 요약하는 효과가 있음.  